# Slice TCA

In [1]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import scipy.ndimage as spnd
import torch
import slicetca
from sklearn.preprocessing import StandardScaler

sys.path.append(r'H:\anthony\repos\NWB_analysis')
from nwb_wrappers import nwb_reader_functions as nwb_read
import src.utils.utils_imaging as imaging_utils
import src.utils.utils_io as io
from src.behavior import compute_performance, plot_single_session

# Set plot parameters.
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'


In [2]:
# Path to the directory containing the processed data.
processed_dir = r"//sv-nas1.rcp.epfl.ch/Petersen-Lab/analysis/Anthony_Renard/data_processed/mice"
nwb_dir = r"//sv-nas1.rcp.epfl.ch/Petersen-Lab/analysis/Anthony_Renard/NWB"

# Session metadata file.
db_path = r"//sv-nas1.rcp.epfl.ch/Petersen-Lab/analysis/Anthony_Renard/mice_info/session_metadata.xlsx"

# Rewarded and non-rewarded NWB files.
group_yaml_rew = r"//sv-nas1.rcp.epfl.ch/Petersen-Lab/analysis/Anthony_Renard/mice_info/groups/imaging_rewarded.yaml"
group_yaml_non_rew = r"//sv-nas1.rcp.epfl.ch/Petersen-Lab/analysis/Anthony_Renard/mice_info/groups/imaging_non_rewarded.yaml"
nwb_list_rew = io.read_group_yaml(group_yaml_rew)
nwb_list_non_rew = io.read_group_yaml(group_yaml_non_rew)
nwb_list = nwb_list_rew + nwb_list_non_rew

In [26]:
# Load data needed to compute before and after learning.

sampling_rate = 30
win = (1, 1.5)  # from stimulus onset to 300 ms after.
win = (int(win[0] * sampling_rate), int(win[1] * sampling_rate))
baseline = (0, 1)
baseline = (int(baseline[0] * sampling_rate), int(baseline[1] * sampling_rate))
n_trials = 20
days = ['-2', '-1', '0', '+1', '+2']
apply_pca = False
variance_to_retain = 0.8

# mouse_list = ['AR127', 'AR133', 'AR135', 'AR143', 'AR144',]
db = io.read_excel_db(db_path)
db = db.loc[
    (db.reward_group=='R-')
            & (db.two_p_imaging=='yes')
            & (db.exclude!='exclude')
            & (db.two_p_exclude!='exclude')]
mouse_list = list(db['subject_id'].unique())

mouse_list = [m for m in mouse_list if m not in ['AR132', 'MI069']]
mouse_list

mouse_list = ['AR127', 'AR133', 'AR135', 'AR143', 'AR144', 'AR131', 'AR137','AR139']

c:\Users\aprenard\Anaconda3\envs\fast-learning\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():


In [27]:
session_list, nwb_files = io.select_sessions_from_db(
    db_path, nwb_dir,
    experimenters=['AR', 'GF', 'MI'],
    exclude_cols=['exclude', 'two_p_exclude'],
    subject_id=mouse_list,
    day=['-2', '-1', '0', '+1', '+2'],
    two_p_imaging='yes')

# act_map = imaging_utils.shape_features_matrix(mouse_list, session_list, processed_dir, 'UM', n_trials)
# # Subtract baselines.
# act_map = act_map - np.nanmean(act_map[:, :, baseline[0]:baseline[1]],
#                                axis=2, keepdims=True)
act_learning = imaging_utils.shape_features_matrix(mouse_list, session_list, processed_dir, 'WH', n_trials)
# Subtract baselines.
act_learning = act_learning - np.nanmean(act_learning[:, :, baseline[0]:baseline[1]],
                               axis=2, keepdims=True)


c:\Users\aprenard\Anaconda3\envs\fast-learning\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():


subject_id ['AR127', 'AR133', 'AR135', 'AR143', 'AR144', 'AR131', 'AR137', 'AR139']
day ['-2', '-1', '0', '+1', '+2']
two_p_imaging yes
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
Trial type 'WH' is not present in the metadata.
dataset shapes
(180, 60, 181)
(144, 60, 181)
(127, 60, 181)
(205, 60, 181)
(111, 60, 181)
(169, 

In [13]:
# from src.behavior import compute_performance, plot_single_session

# nwb_files = [nwb for nwb in nwb_files if 'MI069' in nwb]
# session_list = [s for s in session_list if 'MI069' in s]

# # Add behavior of D0 below.
# f = nwb_files[1]
# session = session_list[1]
# behav_table = nwb_read.get_trial_table(f)
# behav_table = compute_performance(behav_table, session, db_path)

# palette = sns.color_palette()
# plot_single_session(behav_table, session)
# plt.show()

In [28]:

# # min-max scaling for each neuron for values between [0,1]

# data = np.transpose(act_learning, (1,0,2))
# data = spnd.gaussian_filter1d(data.astype('float32'), sigma=2, axis=-1)
# data = np.array([d / d.max() for d in np.array([d - d.min() for d in data])])

# data = data - np.nanmean(data[:, :, baseline[0]:baseline[1]],
#                                axis=2, keepdims=True)

data = np.transpose(act_learning, (1,0,2))
print(data.shape)
data = spnd.gaussian_filter1d(data.astype('float32'), sigma=3, axis=-1)
# data = data - np.mean(data, axis=(0,2), keepdims=True)
data = data / np.std(data, axis=(0,2), keepdims=True)


(60, 1254, 181)


In [29]:
# Save dataset to run on google colab.
save_dir = r"\\sv-nas1.rcp.epfl.ch\Petersen-Lab\analysis\Anthony_Renard\data_processed\tca"
dataset_name = 'wh_learning_whole_population_both_rew_groups.npy'
np.save(os.path.join(save_dir, dataset_name), data)

In [ ]:

# plt.figure()
# for icell in range(50):
#     plt.plot(np.mean(data[:,icell], axis=(0)))
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
data = torch.tensor(data, dtype=torch.float, device=device)
# The tensor is decomposed into 2 trial-, 0 neuron- and 3 time-slicing components.
components, model = slicetca.decompose(data,
                                       number_components=(3,3,3),
                                       positive=True,
                                       learning_rate=5*10**-3,
                                       min_std=10**-3,
                                       max_iter=10000,
                                       seed=0)


In [18]:
torch.device('cpu')

device(type='cpu')

In [29]:
import pickle

model_dir = r"//sv-nas1.rcp.epfl.ch/Petersen-Lab/analysis/Anthony_Renard/analysis_output/tca/models"
model_name = 'model_wh_learning_whole_population_R+.pkl'
# Load the model and components.

device = torch.device('cpu')
with open(os.path.join(model_dir, model_name), 'rb') as f:
    model = torch.load(f, map_location=device, weights_only=False)
    # model = torch.load(f, map_location=torch.device('cpu'))

In [30]:
sns.set_theme(context='talk', style='ticks', palette='deep')
palette = sns.color_palette()
trial_colors = np.array([palette[i] for i in range(3) for _ in range(50)])
axes = slicetca.plot(model,
              variables=('trial', 'neuron', 'time'),
              colors=(trial_colors, None, None), # we only want the trials to be colored
              ticks=(None, None, np.linspace(0,181,4)), # we only want to modify the time ticks
              tick_labels=(None, None, np.linspace(-1,5,4)),
              sorting_indices=(None, None, None),
              quantile=0.99)
plt.gcf().suptitle('z-score')

c:\Users\aprenard\Anaconda3\envs\fast-learning\Lib\site-packages\slicetca\plotting\factors.py:135: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations.
  if tight_layout: fig.tight_layout()


Text(0.5, 0.98, 'z-score')

In [19]:
slicetca.plot_grid(loss_grid, min_ranks=(6, 6, 6))

In [17]:
slicetca.plot

<function slicetca.plotting.factors.plot(model, components: Sequence[Sequence[numpy.ndarray]] = None, variables: Sequence[str] = ('trial', 'neuron', 'time'), colors: Union[Sequence[numpy.ndarray], Sequence[Sequence[float]]] = (None, None, None), sorting_indices: Sequence[numpy.ndarray] = (None, None, None), ticks: Sequence[numpy.ndarray] = (None, None, None), tick_labels: Sequence[numpy.ndarray] = (None, None, None), quantile: float = 0.95, factor_height: int = 2, aspect: str = 'auto', s: int = 10, cmap: str = None, tight_layout: bool = True, dpi: int = 60)>

In [31]:

a = np.array([[1, 10, 100],[1,2,3]])
np.std(a, axis=1, keepdims=True)

array([[44.69899328],
       [ 0.81649658]])